## Problem Statement

### Business Context

The healthcare industry is rapidly evolving, with professionals facing increasing challenges in managing vast volumes of medical data while delivering accurate and timely diagnoses. The need for quick access to comprehensive, reliable, and up-to-date medical knowledge is critical for improving patient outcomes and ensuring informed decision-making in a fast-paced environment.

Healthcare professionals often encounter information overload, struggling to sift through extensive research and data to create accurate diagnoses and treatment plans. This challenge is amplified by the need for efficiency, particularly in emergencies, where time-sensitive decisions are vital. Furthermore, access to trusted, current medical information from renowned manuals and research papers is essential for maintaining high standards of care.

To address these challenges, healthcare centers can focus on integrating systems that streamline access to medical knowledge, provide tools to support quick decision-making, and enhance efficiency. Leveraging centralized knowledge platforms and ensuring healthcare providers have continuous access to reliable resources can significantly improve patient care and operational effectiveness.

**Common Questions to Answer**

**1. Diagnostic Assistance**: "What are the common symptoms and treatments for pulmonary embolism?"

**2. Drug Information**: "Can you provide the trade names of medications used for treating hypertension?"

**3. Treatment Plans**: "What are the first-line options and alternatives for managing rheumatoid arthritis?"

**4. Specialty Knowledge**: "What are the diagnostic steps for suspected endocrine disorders?"

**5. Critical Care Protocols**: "What is the protocol for managing sepsis in a critical care unit?"

### Objective

As an AI specialist, your task is to develop a RAG-based AI solution using renowned medical manuals to address healthcare challenges. The objective is to **understand** issues like information overload, **apply** AI techniques to streamline decision-making, **analyze** its impact on diagnostics and patient outcomes, **evaluate** its potential to standardize care practices, and **create** a functional prototype demonstrating its feasibility and effectiveness.

### Data Description

The **Merck Manuals** are medical references published by the American pharmaceutical company Merck & Co., that cover a wide range of medical topics, including disorders, tests, diagnoses, and drugs. The manuals have been published since 1899, when Merck & Co. was still a subsidiary of the German company Merck.

The manual is provided as a PDF with over 4,000 pages divided into 23 sections.

## Installing and Importing Necessary Libraries and Dependencies

In [35]:
!pip install pymupdf
!pip install sentence-transformers
!pip install langchain
!pip install transformers
!pip install langchain_community
!pip install torch
!pip install psutil
!pip install chromadb
!pip install tiktoken
!pip install python-dotenv
!pip install langchain-huggingface
!pip install huggingface_hub
!pip install sentencepiece accelerate --quiet

In [2]:
#Libraries for processing dataframes,text
import json, os
import tiktoken
import pandas as pd
import fitz
from dotenv import load_dotenv
import requests
import pickle
import chromadb
import numpy as np
import openai
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer

#Libraries for Loading Data, Chunking, Embedding, and Vector Databases
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEndpoint
from langchain.embeddings import HuggingFaceEmbeddings

## Question Answering using LLM

#### Import the dataset from google drive

In [3]:
# Path to the PDF file
from google.colab import drive
drive.mount('/content/drive')
pdf_path = '/content/drive/MyDrive/Colab Notebooks/NLP_MedicalAsst/medical_diagnosis_manual.pdf'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### Loading the model via API from Hugging face

In [4]:
#Load key from Colab for Mistral model
from google.colab import userdata
API_KEY = userdata.get('token_1')

API_URL = "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.3"
HEADERS = {"Authorization": f"Bearer {API_KEY}"}

#### Function to extract text from pdf

In [77]:
#function to extract text from pdf
def extract_text_from_pdf(pdf_path, max_pages=4200):
    """Extracts text from a PDF up to max_pages"""
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(min(len(doc), max_pages)):
        text += doc[page_num].get_text("text") + "\n\n"
    return text[:6000]

context_text = extract_text_from_pdf(pdf_path)

#### Function to generate response

In [78]:
def ask_question(context,question,max_len,temp,top_k,top_p):
  prompt = f"Context: {context}\n\nQ: {question}\n\nA:"
  payload = {
    "inputs": prompt,
    "parameters": {
        "max_length": max_len,
        "temperature": temp,
        "top_k": top_k,
        "top_p": top_p
        }
    }
  response = requests.post(API_URL, headers=HEADERS, json=payload)

  if response.status_code == 200:
    generated_text = response.json()[0]["generated_text"]
    return generated_text.split("A:")[-1].strip()  # Extracts answer part only
  else:
    return f"Error {response.status_code}: {response.json()}"

#### Question Answering

##### Q1 : 'What is the protocol for managing sepsis in a critical care unit?'

In [ ]:
question = "What is the protocol for managing sepsis in a critical care unit?"
answer = ask_question(context_text, question,max_len=128,temp=0.2,top_k=50,top_p=0.9)
print(f"\nQ: {question}\n\nA: {answer}")


Q: What is the protocol for managing sepsis in a critical care unit?

A: The protocol for managing sepsis in a critical care unit typically involves the following steps:

1. Early recognition and prompt treatment: Recognize sepsis early and initiate treatment as soon as possible. This includes identifying the source of infection, measuring vital signs, and assessing organ function.

2. Resuscitation: Administer fluids and oxygen to maintain adequate blood pressure, heart rate, and oxygenation.

3. Antibiotics: Administer broad-spectrum antibiotics as soon as possible, based on the suspected source of infection.

4. Source control: Identify and treat the source of infection, which may involve drainage of abscesses, removal of foreign bodies, or surgical intervention.

5. Vasopressors: If sepsis-induced hypotension is refractory to fluid resuscitation, administer vasopressors to maintain adequate blood pressure.

6. Corticosteroids: In some cases, corticosteroids may be used to reduce i

##### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [ ]:
question = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
answer = ask_question(context_text, question,max_len=128,temp=0.2,top_k=50,top_p=0.9)
print(f"\nQ: {question}\n\nA: {answer}")


Q: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

A: Common symptoms for appendicitis include sudden, sharp pain in the lower right abdomen, nausea, vomiting, loss of appetite, fever, and abdominal swelling. Appendicitis cannot be cured with medicine, and it requires surgical removal of the appendix, known as an appendectomy. The most common surgical procedure for appendicitis is laparoscopic appendectomy, which is performed through small incisions using a camera and surgical instruments. In some cases, an open appendectomy may be necessary, which involves a larger incision.


##### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [ ]:
question = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
answer = ask_question(context_text, question,max_len=128,temp=0.2,top_k=50,top_p=0.9)
print(f"\nQ: {question}\n\nA: {answer}")


Q: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

A: Sudden patchy hair loss, also known as alopecia areata, is an autoimmune disorder that causes hair loss on the scalp, beard, and other areas of the body. The exact cause of alopecia areata is unknown, but it is believed to be related to a problem with the immune system.

There are several treatments available for alopecia areata, including:

1. Corticosteroids: These are anti-inflammatory medications that can help reduce inflammation and promote hair growth. They can be applied topically to the affected area or taken orally.
2. Immunomodulators: These medications work by modifying the immune system to help reduce inflammation and promote hair growth. Examples include minoxidil and anthralin.
3. Light therapy: This involves exposing the affected area to ultraviolet light, which can help stimula

##### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [ ]:
question = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
answer = ask_question(context_text, question,max_len=128,temp=0.2,top_k=50,top_p=0.9)
print(f"\nQ: {question}\n\nA: {answer}")


Q: What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

A: The treatment for a person with a brain injury depends on the severity and location of the injury. Here are some common treatments:

1. Emergency care: Immediate care may include airway management, breathing support, and medication to control increased intracranial pressure or prevent seizures.

2. Surgery: Depending on the location and severity of the injury, surgery may be necessary to remove blood clots, repair skull fractures, or relieve pressure on the brain.

3. Medications: Various medications may be used to manage symptoms such as pain, seizures, and increased intracranial pressure.

4. Rehabilitation: Physical, occupational, and speech therapy may be necessary to help the person regain lost functions and adapt to any permanent impairments.

5. Lifestyle changes: Depending on the extent of the injury, the per

##### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [ ]:
question = "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
answer = ask_question(context_text, question,max_len=128,temp=0.2,top_k=50,top_p=0.9)
print(f"\nQ: {question}\n\nA: {answer}")


Q: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

A: In the event of a leg fracture during a hiking trip, the following steps should be taken:

1. Immediate First Aid: Apply a splint to immobilize the fractured leg and control swelling. If possible, elevate the injured leg above heart level.

2. Transport: Arrange for immediate transport to a medical facility. If professional help is not available, a fellow hiker can carry the injured person on a stretcher or a makeshift litter.

3. Pain Management: Administer pain relief medication, such as acetaminophen or ibuprofen, if available.

4. Medical Attention: Upon arrival at the hospital, the fracture will be assessed and treated by an orthopedic surgeon. Depending on the severity of the fracture, treatment may include closed reduction (manipulation of the bone back into its normal position without surgery), c

#### Observations for LLM Question answering:

The parameters used for 5 questions are = max_len=128,temp=0.2,top_k=50,top_p=0.9


1. **Answer_1** : The answer is clear, concise, and focused, without any unnecessary information for how to handle sepsis in critical care unit.

2. **Answer_2** : The symptoms, cure and surgical methods for Appendicitis is very brief and rather a more explained version would be better.

3. **Answer_3** : The answer alopecia areata was described accurately and all the points needed are present.

4. **Answer_4** : Due to token size the answer for treatments recommended for a person who has sustained a physical injury to brain tissue is very on the point and as expected no additional information is used.

5. **Answer_5** : Since the max_tokens used were 128 the answer for necessary precautions and treatment steps for a person who has fractured their leg is crisp and concised.



**OVERALL:** Using a simple LLM - Mistral-7B-Instruct-v0.3 for Question Answering task and not manipulating many parameters has given concised yet accurate outputs. Maybe including prompt and fine tuning the model may give us a better understanding of the answers.

## Question Answering using LLM with Prompt Engineering

#### Function to generate response with prompt

In [84]:
def ask_question_prompt(context,question,max_len,temp,top_k,top_p):
  prompt = f"""You are an expert AI specializing in medical literature. Your task is to analyze the provided medical textbook content and generate precise, well-structured answers.

**Context:**
{context}

**Instruction:**
- **Understand the given medical content** before answering.
- Provide a **detailed, structured, and step-by-step answer** to the question.
- Use **numbered lists, bullet points, and professional medical terminology**.
- Format the response **clearly** with headings and subheadings where needed.
- Keep the explanation **concise but comprehensive**, avoiding unnecessary repetition.

**Question:**
Q: {question}

**Answer:**
A:

"""
  payload = {
    "inputs": prompt,
    "parameters": {
        "max_length": max_len,
        "temperature": temp,
        "top_k": top_k,
        "top_p": top_p
        }
    }
  response = requests.post(API_URL, headers=HEADERS, json=payload)

  if response.status_code == 200:
    generated_text = response.json()[0]["generated_text"]
    return generated_text.split("A:")[-1].strip()  # Extracts answer part only
  else:
    return f"Error {response.status_code}: {response.json()}"

#### Question Answering with prompt

##### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [ ]:
question = "What is the protocol for managing sepsis in a critical care unit?"
answer = ask_question_prompt(context_text, question,max_len=156,temp=0.20,top_k=50,top_p=0.9)
print(f"\nQ: {question}\n\nA: {answer}")


Q: What is the protocol for managing sepsis in a critical care unit?

A: 1. **Recognition and Diagnosis**
   - Recognize sepsis early by identifying signs and symptoms such as fever, tachycardia, tachypnea, hypotension, and altered mental status.
   - Use the Sequential [Sepsis-related] Organ Failure Assessment (SOFA) score to diagnose sepsis.

2. **Initial Resuscitation**
   - Administer high-flow oxygen and monitor oxygen saturation.
   - Initiate intravenous fluid resuscitation to maintain adequate blood pressure and organ perfusion.
   - Administer broad-spectrum antibiotics as soon as possible.

3. **Hemodynamic Support**
   - Monitor and maintain mean arterial pressure (MAP) ≥ 65 mmHg.
   - Use vasopressors if necessary to maintain MAP.
   - Consider using inotropes to improve cardiac output.

4. **Critical Care Monitoring and Support**
   - Monitor and manage lactate levels, electrolytes, and acid-base balance.
   - Provide mechanical ventilation if necessary to maintain adequa

##### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [ ]:
question = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
answer = ask_question_prompt(context_text, question,max_len=228,temp=0.3,top_k=10,top_p=0.85)
print(f"\nQ: {question}\n\nA: {answer}")


Q: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

A: 1. **Symptoms of Appendicitis**
   - Sudden pain in the lower right abdomen, starting around the navel and then moving to the lower abdomen
   - Loss of appetite
   - Nausea and vomiting
   - Fever (often low-grade at first, but can rise as high as 102°F or 38.9°C)
   - Abdominal swelling
   - Pain worsening with walking or coughing
   - Constipation or diarrhea
   - Inability to pass gas

2. **Treatment of Appendicitis**
   - **Medical Treatment:** Antibiotics may be used to treat appendicitis in some cases, but they are not a substitute for surgery. The appendix can rupture, leading to a potentially life-threatening infection called peritonitis.
   - **Surgical Treatment:** The standard treatment for appendicitis is surgical removal of the appendix, known as an appendectomy. This procedure can be performed as an open appendectomy or 

##### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [ ]:
question = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
answer = ask_question_prompt(context_text, question,max_len=256,temp=0.5,top_k=45,top_p=0.7)
print(f"\nQ: {question}\n\nA: {answer}")


Q: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

A: 1. **Causes of Sudden Patchy Hair Loss**
   - Alopecia Areata: An autoimmune disorder that causes the immune system to attack hair follicles, resulting in round, smooth, and patchy hair loss.
   - Tinea Capitis: A fungal infection of the scalp that can cause circular patches of hair loss, often accompanied by scaling and itching.
   - Traction Alopecia: Hair loss caused by constant pulling or tension on the hair, often seen in individuals who wear tight hairstyles or braids.
   - Stress: Sudden or prolonged stress can cause temporary hair loss, known as telogen effluvium.

2. **Diagnosis**
   - Physical Examination: A doctor will examine the scalp and hair for signs of patchy hair loss, scaling, or other abnormalities.
   - Scalp Biopsy: In some cases, a small sample of scalp tissue may be tak

##### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [85]:
question = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
answer = ask_question_prompt(context_text, question,max_len=256,temp=0.4,top_k=40,top_p=0.75)
print(f"\nQ: {question}\n\nA: {answer}")


Q: What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

A: 1. **Initial Management**
   - **Airway and Breathing:** Ensure the patient's airway is clear, and provide oxygen support if necessary.
   - **Circulation:** Monitor and maintain blood pressure and heart rate.
   - **Disability:** Assess the level of consciousness and neurological status.
   - **Exposure:** Expose the patient to allow for a complete physical examination.

2. **Immediate Interventions**
   - **Neurosurgery:** Depending on the severity and location of the brain injury, surgery may be required to remove hematomas, repair damaged tissue, or relieve pressure on the brain.
   - **Decompressive Craniectomy:** In cases of severe swelling or increased intracranial pressure, a decompressive craniectomy may be performed to create more space for the swollen brain.

3. **Medications**
   - **Anti-seizure Medicati

##### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [86]:
question = "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
answer = ask_question_prompt(context_text, question,max_len=256,temp=0.1,top_k=50,top_p=0.9)
print(f"\nQ: {question}\n\nA: {answer}")


Q: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

A: 1. **Initial Assessment and Stabilization**
   - Assess the severity of the fracture and any associated injuries.
   - Immobilize the affected leg using a splint or a cast to prevent further damage and alleviate pain.
   - Administer pain relief medication as needed.
   - Monitor vital signs, including pulse, blood pressure, and respiratory rate.

2. **Transportation**
   - If the patient is unable to walk, they should be transported to the nearest medical facility using a stretcher or a wheelchair.
   - In remote areas, a helicopter or a four-wheel drive vehicle may be necessary for transportation.

3. **Hospital Admission and Diagnostics**
   - Upon arrival at the hospital, the patient will undergo further evaluation, including X-rays, CT scans, or MRI to confirm the fracture and determine the best tre

#### Observations for LLM Question answering with PROMPT declaration and fine tuning for each query:


1. **Answer_1** (max_len=156,temp=0.20,top_k=50,top_p=0.9) : The answer with fine tuning yet using prompt has given a better response for how to handle sepsis in critical care unit with detailed steps.

2. **Answer_2** (max_len=228,temp=0.3,top_k=10,top_p=0.85) : The symptoms, cure and surgical methods for Appendicitis is very detailed and note mentioned is extra additive to the output.

3. **Answer_3** (max_len=256,temp=0.5,top_k=45,top_p=0.7) : The answer alopecia areata was described accurately and all the points needed are present with lot of detailing.

4. **Answer_4** (max_len=256,temp=0.4,top_k=40,top_p=0.75) : The answer for a person who has sustained a physical injury to brain tissue is very on the point and as expected structured.

5. **Answer_5** (max_len=256,temp=0.1,top_k=50,top_p=0.9) : The answer for necessary precautions and treatment steps for a person who has fractured their leg is detailed and structured.



**OVERALL:** Using a simple LLM - Mistral-7B-Instruct-v0.3 for Question Answering including prompt and fine tuning the model has given us a better understanding of the answers with detailed outputs and additional note pointers.

## Data Preparation for RAG

### Loading the Data

In [5]:
pdf_loader = PyMuPDFLoader(pdf_path)
Medical_data = pdf_loader.load()

### Data Overview

#### Checking the first 5 pages

In [ ]:
for i in range(5):
    print(f"Page Number : {i+1}",end="\n")
    print(Medical_data[i].page_content,end="\n")

Page Number : 1
vaddi.rithika@gmail.com
8B1FOZKCHQ
eant for personal use by vaddi.rithika@gm
shing the contents in part or full is liable
Page Number : 2
vaddi.rithika@gmail.com
8B1FOZKCHQ
This file is meant for personal use by vaddi.rithika@gmail.com only.
Sharing or publishing the contents in part or full is liable for legal action.
Page Number : 3
Table of Contents
1
Front    ................................................................................................................................................................................................................
1
Cover    .......................................................................................................................................................................................................
2
Front Matter    ....................................................................................................................................................................................

#### Checking the number of pages

In [ ]:
len(Medical_data)

4114

#### Save and load pickle file to avoid running the code again

In [6]:
# Save data using pickle
def save_pickle(data, filename):
    with open(filename, "wb") as f:
        pickle.dump(data, f)

# Load data using pickle
def load_pickle(filename):
    if os.path.exists(filename):
        with open(filename, "rb") as f:
            return pickle.load(f)
    return None

#### Extract text from pdf

In [7]:
# Load PDF and extract text
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = "\n".join([page.get_text("text") for page in doc])
    return text

#### Data Chunking

In [8]:
# Chunking the text using RecursiveCharacterTextSplitter
def chunk_text(text):
    # Initialize the text splitter with the tiktoken encoder
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(encoding_name='cl100k_base', chunk_size=512, chunk_overlap=20)
    # Load and split the PDF into chunks
    document_chunks = pdf_loader.load_and_split(text_splitter)
    return document_chunks

#### Generate embeddings

In [9]:
# Generate and store embeddings
def generate_embeddings(chunks, model):
    texts = [doc.page_content for doc in chunks]  # Extract text from Document objects
    embeddings = [model.encode(text).tolist() for text in texts]
    return embeddings

#### Load and store from ChromaDB

In [79]:
# Store in ChromaDB using Chroma.from_documents
def store_in_chromadb(chunks, embedding_model, persist_directory):
    vectorstore = Chroma.from_documents(
        documents=chunks,  # Chunks are now Document objects
        embedding=embedding_model,
        persist_directory=persist_directory
    )
    vectorstore.persist()
    return vectorstore

# Load ChromaDB
def load_chromadb(embedding_model, persist_directory):
  if not os.path.exists(persist_directory) or not os.listdir(persist_directory):
      print("[INFO] ChromaDB is empty. Initializing storage.")
      return None
  return Chroma(persist_directory=persist_directory, embedding_function=embedding_model)

#### Retrieve the relevant chunks

In [11]:
# Retrieve relevant chunks
def retrieve_relevant_chunks(query, vectorstore, top_k=3):
  if vectorstore is None:
    return ["No data available in ChromaDB."]
  docs = vectorstore.similarity_search(query, k=top_k)
  return [doc.page_content for doc in docs]

#### System and User Prompt Template

In [12]:
qna_system_message = """
You are a medical AI. Answer the user's question using ONLY the given context.
Do not make up facts. If the answer isn't in the context, reply: " "\"The provided document does not contain sufficient information.\" "
Be clear, concise, and medically accurate.
"""

In [13]:
qna_user_message_template = """
###Context
Here are some documents that are relevant to the question mentioned below.
{context}

###Question
{question}
"""

#### Generate response for RAG model

In [14]:
# Generate answer using Mistral-7B API
def generate_answer(query,vectorstore,max_tokens,temp,top_p,top_k):

    global qna_system_message, qna_user_message_template

    retrieved_chunks = retrieve_relevant_chunks(query, vectorstore)
    context = " ".join(retrieved_chunks)

    # Construct user message using the template
    user_message = qna_user_message_template.replace("{context}", context).replace("{question}", query)

    # Final prompt includes both system message and user message
    prompt = f"{qna_system_message}\n\n{user_message}"

    payload = {
        "inputs": prompt,
        "parameters":
         {
        "max_tokens": max_tokens,
        "temperature": temp,
        "top_k": top_k,
        "top_p": top_p
        }
    }
    response = requests.post(API_URL, headers=HEADERS, json=payload)

    # Handle the API response
    if response.status_code == 200:
        response_json = response.json()
        if isinstance(response_json, list) and len(response_json) > 0 and "generated_text" in response_json[0]:
            return response_json[0]["generated_text"].strip()
        else:
            return f"Unexpected response format: {response_json}"
    elif response.status_code == 402:
        return "API quota exceeded. Please upgrade your plan or try a local model."
    else:
        return f"Error: {response.status_code} - {response.text}"

### Main Execution of the RAG model is done here

In [15]:
# File paths
PDF_PATH = "/content/drive/MyDrive/Colab Notebooks/NLP_MedicalAsst/medical_diagnosis_manual.pdf"
CHUNKS_FILE = "/content/drive/MyDrive/Colab Notebooks/NLP_MedicalAsst/document_chunks.pkl"
EMBEDDINGS_FILE = "/content/drive/MyDrive/Colab Notebooks/NLP_MedicalAsst/embeddings.pkl"
CHROMA_DB_PATH = "/content/drive/MyDrive/Colab Notebooks/NLP_MedicalAsst/chroma_db"

# Main execution
if __name__ == "__main__":
    # Load PDF and process text
    pdf_text = extract_text_from_pdf(PDF_PATH)

    # Load or create chunks
    chunks = load_pickle(CHUNKS_FILE) or chunk_text(pdf_text)
    save_pickle(chunks, CHUNKS_FILE)

    # Load embedding model
    embedding_model = HuggingFaceEmbeddings(model_name="thenlper/gte-large")

    # Load or create embeddings
    embeddings = load_pickle(EMBEDDINGS_FILE) or generate_embeddings(chunks, embedding_model)
    save_pickle(embeddings, EMBEDDINGS_FILE)

    # Load vector store
    vectorstore = load_chromadb(embedding_model, CHROMA_DB_PATH)

    # If ChromaDB is empty, create and store chunks
    if vectorstore is None:
      print("[INFO] Creating a new ChromaDB vector store...")
      vectorstore = store_in_chromadb(chunks, embedding_model, CHROMA_DB_PATH)

<ipython-input-15-39c5286969a0>:17: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="thenlper/gte-large")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  war

modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.9k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

<ipython-input-10-f5649ec1381e>:16: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  return Chroma(persist_directory=persist_directory, embedding_function=embedding_model)


## Question Answering using RAG

##### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [ ]:
query = "What is the protocol for managing sepsis in a critical care unit?"
answer = generate_answer(query, vectorstore,256,0.2,0.9,50)
print(answer)

You are an expert AI specializing in medical literature. Your task is to analyze the provided medical textbook content and generate precise, well-structured answers.
User input will contain relevant context from the document to help you generate precise responses.

- The context will be marked with the token: **### Context**
- The user’s question will be marked with the token: **### Question**

### Instructions:
1. Answer **only** using the provided context. **Do not generate information beyond what is given.**
2. Do **not** mention the context explicitly in your response. Provide a natural, well-structured answer.
3. If the answer **is not found** in the context, respond with: **"The provided document does not contain sufficient information."**
4. Ensure your responses are **clear, medically accurate, and concise**.



###Context
Here are some documents that are relevant to the question mentioned below.
16 - Critical Care Medicine
Chapter 222. Approach to the Critically Ill Patient
In

##### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [ ]:
query = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
answer = generate_answer(query, vectorstore,256,0.2,0.9,50)
print(answer)

You are an expert AI specializing in medical literature. Your task is to analyze the provided medical textbook content and generate precise, well-structured answers.
User input will contain relevant context from the document to help you generate precise responses.

- The context will be marked with the token: **### Context**
- The user’s question will be marked with the token: **### Question**

### Instructions:
1. Answer **only** using the provided context. **Do not generate information beyond what is given.**
2. Do **not** mention the context explicitly in your response. Provide a natural, well-structured answer.
3. If the answer **is not found** in the context, respond with: **"The provided document does not contain sufficient information."**
4. Ensure your responses are **clear, medically accurate, and concise**.



###Context
Here are some documents that are relevant to the question mentioned below.
• Surgical removal
• IV fluids and antibiotics
Treatment of acute appendicitis is 

##### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [ ]:
query = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
answer = generate_answer(query, vectorstore,256,0.2,0.9,50)
print(answer)

You are an expert AI specializing in medical literature. Your task is to analyze the provided medical textbook content and generate precise, well-structured answers.
User input will contain relevant context from the document to help you generate precise responses.

- The context will be marked with the token: **### Context**
- The user’s question will be marked with the token: **### Question**

### Instructions:
1. Answer **only** using the provided context. **Do not generate information beyond what is given.**
2. Do **not** mention the context explicitly in your response. Provide a natural, well-structured answer.
3. If the answer **is not found** in the context, respond with: **"The provided document does not contain sufficient information."**
4. Ensure your responses are **clear, medically accurate, and concise**.



###Context
Here are some documents that are relevant to the question mentioned below.
corticosteroids, retinoids, or immunosuppressants.
Hair loss due to chemotherapy i

##### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [ ]:
query = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
answer = generate_answer(query, vectorstore,256,0.2,0.9,50)
print(answer)

You are an expert AI specializing in medical literature. Your task is to analyze the provided medical textbook content and generate precise, well-structured answers.
User input will contain relevant context from the document to help you generate precise responses.

- The context will be marked with the token: **### Context**
- The user’s question will be marked with the token: **### Question**

### Instructions:
1. Answer **only** using the provided context. **Do not generate information beyond what is given.**
2. Do **not** mention the context explicitly in your response. Provide a natural, well-structured answer.
3. If the answer **is not found** in the context, respond with: **"The provided document does not contain sufficient information."**
4. Ensure your responses are **clear, medically accurate, and concise**.



###Context
Here are some documents that are relevant to the question mentioned below.
Chapter 324. Traumatic Brain Injury
Introduction
Traumatic brain injury (TBI) is p

##### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [ ]:
query = "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery??"
answer = generate_answer(query, vectorstore,256,0.2,0.9,50)
print(answer)

You are an expert AI specializing in medical literature. Your task is to analyze the provided medical textbook content and generate precise, well-structured answers.
User input will contain relevant context from the document to help you generate precise responses.

- The context will be marked with the token: **### Context**
- The user’s question will be marked with the token: **### Question**

### Instructions:
1. Answer **only** using the provided context. **Do not generate information beyond what is given.**
2. Do **not** mention the context explicitly in your response. Provide a natural, well-structured answer.
3. If the answer **is not found** in the context, respond with: **"The provided document does not contain sufficient information."**
4. Ensure your responses are **clear, medically accurate, and concise**.



###Context
Here are some documents that are relevant to the question mentioned below.
batting, and layers 2 and 4 are elastic bandages. The injured limb is elevated abo

### Function to generate response for a fine tuned model

In [70]:
# Generate answer using Mistral-7B API
def generate_answer_finetune(query,vectorstore,max_tokens,temp,top_p,top_k):

    global qna_system_message, qna_user_message_template

    retrieved_chunks = retrieve_relevant_chunks(query, vectorstore)
    context = " ".join(retrieved_chunks)

    # Construct user message using the template
    user_message = qna_user_message_template.replace("{context}", context).replace("{question}", query)

    # Final prompt includes both system message and user message
    prompt = f"{qna_system_message}\n\n{user_message}"

    payload = {
        "inputs": prompt,
        "parameters":
         {
        "max_tokens": max_tokens,
        "temperature": temp,
        "top_k": top_k,
        "top_p": top_p
        }
    }
    response = requests.post(API_URL, headers=HEADERS, json=payload)

    # Handle the API response
    if response.status_code == 200:
        response_json = response.json()
        if isinstance(response_json, list) and len(response_json) > 0 and "generated_text" in response_json[0]:
            return response_json[0]["generated_text"].strip()
        else:
            return f"Unexpected response format: {response_json}"
    elif response.status_code == 402:
        return "API quota exceeded. Please upgrade your plan or try a local model."
    else:
        return f"Error: {response.status_code} - {response.text}"

### Fine-tuning with RAG

##### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [71]:
question = "What is the protocol for managing sepsis in a critical care unit?"
answer = generate_answer_finetune(question, vectorstore,156,0.2,0.9,50)
print(answer)

You are a medical AI. Answer the user's question using ONLY the given context.
Do not make up facts. If the answer isn't in the context, reply: " ""The provided document does not contain sufficient information." "
Be clear, concise, and medically accurate.



###Context
Here are some documents that are relevant to the question mentioned below.
16 - Critical Care Medicine
Chapter 222. Approach to the Critically Ill Patient
Introduction
Critical care medicine specializes in caring for the most seriously ill patients. These patients are best
treated in an ICU staffed by experienced personnel. Some hospitals maintain separate units for special
populations (eg, cardiac, surgical, neurologic, pediatric, or neonatal patients). ICUs have a high
nurse:patient ratio to provide the necessary high intensity of service, including treatment and monitoring
of physiologic parameters.
Supportive care for the ICU patient includes provision of adequate nutrition (see p. 21) and prevention of
infection, s

##### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [72]:
question = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
answer = generate_answer_finetune(question, vectorstore,228,0.3,0.85,10)
print(answer)

You are a medical AI. Answer the user's question using ONLY the given context.
Do not make up facts. If the answer isn't in the context, reply: " ""The provided document does not contain sufficient information." "
Be clear, concise, and medically accurate.



###Context
Here are some documents that are relevant to the question mentioned below.
• Surgical removal
• IV fluids and antibiotics
Treatment of acute appendicitis is open or laparoscopic appendectomy; because treatment delay
increases mortality, a negative appendectomy rate of 15% is considered acceptable. The surgeon can
usually remove the appendix even if perforated. Occasionally, the appendix is difficult to locate: In these
cases, it usually lies behind the cecum or the ileum and mesentery of the right colon. A contraindication to
appendectomy is inflammatory bowel disease involving the cecum. However, in cases of terminal ileitis
and a normal cecum, the appendix should be removed.
Appendectomy should be preceded by IV antib

##### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [80]:
question = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
answer = generate_answer_finetune(question, vectorstore,256,0.5,0.7,45)
print(answer)

You are a medical AI. Answer the user's question using ONLY the given context.
Do not make up facts. If the answer isn't in the context, reply: " ""The provided document does not contain sufficient information." "
Be clear, concise, and medically accurate.



###Context
Here are some documents that are relevant to the question mentioned below.
corticosteroids, retinoids, or immunosuppressants.
Hair loss due to chemotherapy is temporary and is best treated with a wig; when hair regrows, it may be
different in color and texture from the original hair. Hair loss due to telogen effluvium or anagen effluvium
is usually temporary as well and abates after the precipitating agent is eliminated.
Key Points
• Androgenetic alopecia (male-pattern and female-pattern hair loss) is the most common type of hair loss.
• Concomitant virilization in women or scarring hair loss should prompt a thorough evaluation for the
underlying disorder.
• Microscopic hair examination or scalp biopsy may be required f

##### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [81]:
question = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
answer = generate_answer_finetune(question, vectorstore,256,0.4,0.75,40)
print(answer)

You are a medical AI. Answer the user's question using ONLY the given context.
Do not make up facts. If the answer isn't in the context, reply: " ""The provided document does not contain sufficient information." "
Be clear, concise, and medically accurate.



###Context
Here are some documents that are relevant to the question mentioned below.
Chapter 324. Traumatic Brain Injury
Introduction
Traumatic brain injury (TBI) is physical injury to brain tissue that temporarily or permanently
impairs brain function. Diagnosis is suspected clinically and confirmed by imaging (primarily
CT). Initial treatment consists of ensuring a reliable airway and maintaining adequate
ventilation, oxygenation, and blood pressure. Surgery is often needed in patients with more
severe injury to place monitors to track and treat intracranial pressure, decompress the brain if
intracranial pressure is increased, or remove intracranial hematomas. In the first few days after
the injury, maintaining adequate brain p

##### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [82]:
question = "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
answer = generate_answer_finetune(question, vectorstore,256,0.1,0.9,50)
print(answer)

You are a medical AI. Answer the user's question using ONLY the given context.
Do not make up facts. If the answer isn't in the context, reply: " ""The provided document does not contain sufficient information." "
Be clear, concise, and medically accurate.



###Context
Here are some documents that are relevant to the question mentioned below.
batting, and layers 2 and 4 are elastic bandages. The injured limb is elevated above the heart for the first
2 days in a position that allows gravity to help drain edema fluid and thus minimize swelling. After 48 h,
periodic application of warmth (eg, a heating pad) for 15 to 20 min may relieve pain and speed healing.
Immobilization: Immobilization decreases pain and facilitates healing by preventing further injury and is
helpful except for very rapidly healing injuries. Joints proximal and distal to the injury should be
immobilized.
A cast is usually used for fractures or other injuries that require weeks of immobilization. Rarely, swelling
unde

#### Observations for RAG Question answering with PROMPT declaration and fine tuning for each query vs Default parameters RAG Question answering:

**Deafult Parameters** : max_len=256,temp=0.2,top_k=50,top_p=0.9

1. **Answer_1** (max_len=156,temp=0.20,top_k=50,top_p=0.9) : The answer with fine tuning yet using prompt has given a better response for how to handle sepsis in critical care unit with detailed steps compared to when using default parameters.

2. **Answer_2** (max_len=228,temp=0.3,top_k=10,top_p=0.85) : The symptopms, cure and surgical methods for Appendicitis is concised information for both default parameters and fine tuning due to parameter configuration.

3. **Answer_3** (max_len=256,temp=0.5,top_k=45,top_p=0.7) : The answer alopecia areata was described accurately and all the points needed are concised information for both default parameters and fine tuning due to parameter configuration.

4. **Answer_4** (max_len=256,temp=0.4,top_k=40,top_p=0.75) : Due to token size the answer for treatments recommended for a person who has sustained a physical injury to brain tissue is very on the point and as expected no additional information is used even with prompt.  

5. **Answer_5** (max_len=256,temp=0.1,top_k=50,top_p=0.9) : The answer for necessary precautions and treatment steps for a person who has fractured their leg is crisp and concised and same as before.


**OVERALL:** Using a simple RAG model with model - Mistral-7B-Instruct-v0.3 for Question Answering including prompt and fine tuning the model has given us a better understanding of the answers with detailed outputs and additional note pointers compared to deafult paramaters based on combination.

## Output Evaluation

#### Defining the Evaluation Prompts

In [76]:
groundedness_rater_system_message = """You are an expert evaluator. Your job is to assess how grounded an answer is in the provided context.
Return a JSON object with the following keys:
{
  "score": <integer from 1 to 5>,
  "reason": "<brief explanation (1-2 sentences)>",
  "hallucination": true/false
}
ONLY return valid JSON. Do not include markdown, explanations, or anything outside the JSON."""

In [62]:
relevance_rater_system_message = """You are an expert evaluator. Your task is to rate how relevant the model's answer is to the user's original question, based on the provided context.

Return ONLY a JSON object in the following format:

{
  "score": <integer from 1 to 5>,
  "reason": "<brief explanation (1-2 sentences)>",
  "answer_on_topic": true or false
}

Only consider information found in the context. Do NOT include any output other than raw JSON."""

In [54]:
user_message_template =""""
### Context:
{context}

### Question:
{question}
"""

#### Defining the Evaluation Function

In [63]:
from google.colab import userdata

def generate_groundedness_relevance_response(query,vectorstore,max_tokens,temp,top_p,top_k):

  global qna_system_message,qna_user_message_template
  global groundedness_rater_system_message, relevance_rater_system_message

  retrieved_chunks = retrieve_relevant_chunks(query, vectorstore)
  context = " ".join(retrieved_chunks[:2])

  HF_Key=userdata.get('token_2')
  eval_API_URL ="https://api-inference.huggingface.co/models/meta-llama/Llama-2-7b-chat-hf"
  HEADERS1 = {"Authorization": f"Bearer {HF_Key}"}

  # Construct user message using the template
  user_message = qna_user_message_template.replace("{context}", context).replace("{question}", query)
  # Final prompt includes both system message and user message
  prompt = f"{qna_system_message}\n\n{user_message}"

  # Generate the Mistral LLM's response using Hugging Face API
  payload = {"inputs": prompt,"parameters":{"max_length": max_tokens,"temperature": temp,"top_k": top_k,"top_p": top_p} }
  generated_text = requests.post(API_URL, headers=HEADERS, json=payload)
  generated_response = generated_text.json()[0]["generated_text"] if generated_text.status_code == 200 else f"Error: {generated_text.status_code}"

  # Groundedness evaluation prompt
  groundedness_prompt = f"""[INST] {groundedness_rater_system_message}
  ### User Question:
  {query}
  ### Mistral LLM's response:
  {generated_response}
  ### Relevant Excerpts from the Medical Manual:
  {context}
  [/INST]"""

  # Relevance evaluation prompt
  relevance_prompt = f"""[INST] {relevance_rater_system_message}
  ### User Question:
  {query}
  ### Mistral LLM's response:
  {generated_response}
  ### Relevant Excerpts from the Medical Manual:{context}
  [/INST]"""

  def evaluate(prompt):
    payload = {"inputs": prompt,"parameters": {"max_length": max_tokens,"temperature": temp,"top_k": top_k,"top_p": top_p}}
    response = requests.post(eval_API_URL, headers=HEADERS1, json=payload)
    if response.status_code == 200:
      return response.json()[0]["generated_text"]
    else:
      return f"Error: {response.status_code}"

  groundedness_evaluation = evaluate(groundedness_prompt)
  relevance_evaluation = evaluate(relevance_prompt)

  # Return structured output
  return {"Groundedness Evaluation": groundedness_evaluation,"Relevance Evaluation": relevance_evaluation }

#### Question Answering

In [64]:
query = "What is the protocol for managing sepsis in a critical care unit?"
answer = generate_groundedness_relevance_response(query,vectorstore,256,0.2,0.9,50)
print(answer)

{'Groundedness Evaluation': '[INST] You are an expert evaluator. Your job is to assess how grounded an answer is in the provided context.\nReturn a JSON object with the following keys:\n{\n  "score": <integer from 1 to 5>,\n  "reason": "<brief explanation (1-2 sentences)>",\n  "hallucination": true/false\n}\nONLY return valid JSON. Do not include markdown, explanations, or anything outside the JSON.\n  ### User Question:\n  What is the protocol for managing sepsis in a critical care unit?\n  ### Mistral LLM\'s response:\n  \nYou are a medical AI. Answer the user\'s question using ONLY the given context.\nDo not make up facts. If the answer isn\'t in the context, reply: " ""The provided document does not contain sufficient information." "\nBe clear, concise, and medically accurate.\n\n\n\n###Context\nHere are some documents that are relevant to the question mentioned below.\n16 - Critical Care Medicine\nChapter 222. Approach to the Critically Ill Patient\nIntroduction\nCritical care med

In [65]:
query = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
answer = generate_groundedness_relevance_response(query, vectorstore,256,0.2,0.9,50)
print(answer)

{'Groundedness Evaluation': '[INST] You are an expert evaluator. Your job is to assess how grounded an answer is in the provided context.\nReturn a JSON object with the following keys:\n{\n  "score": <integer from 1 to 5>,\n  "reason": "<brief explanation (1-2 sentences)>",\n  "hallucination": true/false\n}\nONLY return valid JSON. Do not include markdown, explanations, or anything outside the JSON.\n  ### User Question:\n  What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?\n  ### Mistral LLM\'s response:\n  \nYou are a medical AI. Answer the user\'s question using ONLY the given context.\nDo not make up facts. If the answer isn\'t in the context, reply: " ""The provided document does not contain sufficient information." "\nBe clear, concise, and medically accurate.\n\n\n\n###Context\nHere are some documents that are relevant to the question mentioned below.\n• Surgical removal\n• IV fluids an

In [66]:
query = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
answer = generate_groundedness_relevance_response(query, vectorstore,256,0.2,0.9,50)
print(answer)

{'Groundedness Evaluation': '[INST] You are an expert evaluator. Your job is to assess how grounded an answer is in the provided context.\nReturn a JSON object with the following keys:\n{\n  "score": <integer from 1 to 5>,\n  "reason": "<brief explanation (1-2 sentences)>",\n  "hallucination": true/false\n}\nONLY return valid JSON. Do not include markdown, explanations, or anything outside the JSON.\n  ### User Question:\n  What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?\n  ### Mistral LLM\'s response:\n  \nYou are a medical AI. Answer the user\'s question using ONLY the given context.\nDo not make up facts. If the answer isn\'t in the context, reply: " ""The provided document does not contain sufficient information." "\nBe clear, concise, and medically accurate.\n\n\n\n###Context\nHere are some documents that are relevant to the question mentioned

In [67]:
query = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
answer = generate_groundedness_relevance_response(query, vectorstore,256,0.2,0.9,50)
print(answer)

{'Groundedness Evaluation': '[INST] You are an expert evaluator. Your job is to assess how grounded an answer is in the provided context.\nReturn a JSON object with the following keys:\n{\n  "score": <integer from 1 to 5>,\n  "reason": "<brief explanation (1-2 sentences)>",\n  "hallucination": true/false\n}\nONLY return valid JSON. Do not include markdown, explanations, or anything outside the JSON.\n  ### User Question:\n  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?\n  ### Mistral LLM\'s response:\n  \nYou are a medical AI. Answer the user\'s question using ONLY the given context.\nDo not make up facts. If the answer isn\'t in the context, reply: " ""The provided document does not contain sufficient information." "\nBe clear, concise, and medically accurate.\n\n\n\n###Context\nHere are some documents that are relevant to the question mentioned below.\nChapter 324. Tr

In [68]:
query = "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery??"
answer = generate_groundedness_relevance_response(query, vectorstore,256,0.2,0.9,50)
print(answer)

{'Groundedness Evaluation': '[INST] You are an expert evaluator. Your job is to assess how grounded an answer is in the provided context.\nReturn a JSON object with the following keys:\n{\n  "score": <integer from 1 to 5>,\n  "reason": "<brief explanation (1-2 sentences)>",\n  "hallucination": true/false\n}\nONLY return valid JSON. Do not include markdown, explanations, or anything outside the JSON.\n  ### User Question:\n  What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery??\n  ### Mistral LLM\'s response:\n  \nYou are a medical AI. Answer the user\'s question using ONLY the given context.\nDo not make up facts. If the answer isn\'t in the context, reply: " ""The provided document does not contain sufficient information." "\nBe clear, concise, and medically accurate.\n\n\n\n###Context\nHere are some documents that are relevant to the question mentioned below.\nb

#### Observations for evaulating the responses from Mistral-7B-Instruct-v0.3 using the LLM model Llama-2-7b-chat-hf:

The Default parameters used are max_len=256,temp=0.2,top_k=50,top_p=0.9

**Answer_1** :

**Groundedness Evaluation:**

*   *score* - 3
*   *hallucination*: false
*   *Reason* - The provided document does not contain sufficient information about the protocol for managing sepsis in a critical care unit. The document provides general information about critical care medicine and the monitoring and testing of critically ill patients, but it does not provide specific guidance on how to manage sepsis.

**Relevance Evaluation:**

*   *score* - 4
*   *answer_on_topic*: No
*   *Reason* -  The provided document does not contain sufficient information about the protocol for managing sepsis in a critical care unit. The document provides general information about critical care medicine and the monitoring and testing of critically ill patients, but it does not provide specific guidance on managing sepsis.


**Answer_2** :

**Groundedness Evaluation:**

*   *score* - 4
*   *hallucination*: false
*   *Reason* - The answer provides clear and concise information about the common symptoms of appendicitis and the treatment options available. The answer is based on the provided medical context and does not contain any irrelevant or misleading information.The answer directly addresses the user's question and provides relevant information about the symptoms and treatment of appendicitis. The answer is well-structured and easy to understand, with clear explanations and examples. The answer also provides additional context and explanations where necessary, such as the classification of abdominal hernias. Overall, the answer is medically accurate and provides a good assessment of the user's question.

**Relevance Evaluation:**

*   *score* - 4
*   *answer_on_topic*: True
*   *Reason* -  The provided document contains sufficient information to answer the user's question.


**Answer_3** :

**Groundedness Evaluation:**

*   *score* - 4
*   *hallucination*: false
*   *Reason* - The provided context contains relevant information about hair loss and its possible causes. The user's question is clear and specific, and the provided documents provide useful information about the different types of hair loss and their treatments. However, the context does not provide enough information to definitively answer the user's question about the possible causes of sudden patchy hair loss. Therefore, the score is 4, indicating a good effort but with some limitations in the provided information.

**Relevance Evaluation:**

*   *score* - 4
*   *answer_on_topic*: True
*   *Reason* - The provided document contains relevant information related to the user's question. The answer provided is based on the information provided in the context and is medically accurate.


**Answer_4** :

**Groundedness Evaluation:**

*   *score* - 4
*   *hallucination*: false
*   *Reason* - The answer provided is a direct copy-paste from the provided medical manual, with no additional information or context provided. Therefore, the answer is considered a valid response based on the provided context.The answer provided does not contain any hallucinations or made-up information. It is a direct copy-paste from the provided medical manual, which is a reliable and trustworthy source of information. Therefore, the answer is considered valid and free of hallucinations.

**Relevance Evaluation:**

*   *score* - 4
*   *answer_on_topic*: Yes
*   *Reason* - The provided document provides clear and concise information on the treatment of traumatic brain injury (TBI). It covers the initial management of TBI, including ensuring a reliable airway, maintaining adequate ventilation, oxygenation, and blood pressure, and the need for surgery in more severe cases. The document also discusses the importance of preventing complications of altered sensorium and the subsequent need for rehabilitation. The information provided is medically accurate and based on the context given in the question.

**Answer_5** :

**Groundedness Evaluation:**

*   *score* - 4
*   *hallucination*: false
*   *Reason* - The provided document provides sufficient information for the user to understand the necessary precautions and treatment steps for a person with a fractured leg during a hiking trip. The document highlights the importance of elevating the injured limb, applying warmth, and immobilizing the affected area to prevent further injury and promote healing. It also mentions the use of a cast for fractures that require weeks of immobilization. However, the document does not provide specific first aid measures for a fractured leg during a hiking trip, such as splinting or transporting the person, which may be important to consider.

**Relevance Evaluation:**

*   *score* - 5
*   *answer_on_topic*: true
*   *Reason* - The provided document provides comprehensive and accurate information on the necessary precautions and treatment steps for a person with a fractured leg during a hiking trip. The document covers topics such as elevation, warmth application, immobilization, cast usage, and patient instructions. It also highlights the importance of seeking professional medical help as soon as possible. The document is well-structured and easy to understand, making it a valuable resource for anyone in need of information on this topic. Therefore, I have given a score of 5 out of 5.

## Actionable Insights and Business Recommendations

1. Evaluation of the responses shows that it is highly groudeded and relevant to the input given .
2. The default parameters proves to be the best performing.
3. Using prompts gives us structured output from the model and precise to the question.
4. Healthcare centres can deploy these kind of RAG based models in analysing large documents quickly and efficiently.
5. Prompts given must be formatted and apt to the context of the data for getting better results.
6. Establish a feedback loop to fine-tune parameters, improving performance for diverse query types.
7. Continuously adjust RAG parameters based on specific use cases for optimal performance.
8. Prioritize groundedness and relevance in evaluations to ensure reliable and contextually accurate outputs.





In [97]:
!pip install jupyterlab_widgets
!pip install nbconvert

!jupyter nbconvert --to html --template classic --no-input /content/NLP_MedAsst_Rithika.ipynb

[NbConvertApp] Converting notebook /content/NLP_MedAsst_Rithika.ipynb to html
Traceback (most recent call last):
  File "/usr/local/bin/jupyter-nbconvert", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/jupyter_core/application.py", line 283, in launch_instance
    super().launch_instance(argv=argv, **kwargs)
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/nbconvert/nbconvertapp.py", line 420, in start
    self.convert_notebooks()
  File "/usr/local/lib/python3.11/dist-packages/nbconvert/nbconvertapp.py", line 597, in convert_notebooks
    self.convert_single_notebook(notebook_filename)
  File "/usr/local/lib/python3.11/dist-packages/nbconvert/nbconvertapp.py", line 563, in convert_single_notebook
    output, resources = self.export_single_notebook(
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^